In [2]:
!pip3 install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 4.5 MB/s eta 0:00:00 MB/s eta 0:00:01


In [4]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
import configparser as cp
#https://github.com/Refinitiv-API-Samples/Article.EikonAPI.Python.NewsSentimentAnalysis

In [5]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

In [6]:
from datetime import datetime, timedelta
end = datetime.today()
start = end - timedelta(days=(365.2*4))
print(start, end)

2019-04-11 23:09:06.693526 2023-04-11 18:21:06.693526


In [15]:
df = ek.get_news_headlines('R:SAP.N AND Language:LEN', date_to = end, count=100)
df.head()

,versionCreated,text,storyId,sourceCode
2023-04-10 16:57:07.252,2023-04-10 16:57:07.252000+00:00,Fitch Assigns Qualtrics International Inc. Fir...,urn:newsml:newswire.refinitiv.com:20230410:nFI...,NS:FCH
2023-04-07 02:55:46.751,2023-04-07 02:55:46.751000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA
2023-04-06 14:05:02.721,2023-04-06 14:05:02.721000+00:00,Augmentir Brings AI to SAP Plant Maintenance w...,urn:newsml:newswire.refinitiv.com:20230406:nPn...,NS:PRN
2023-04-06 03:18:30.895,2023-04-06 03:18:30.895000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230406:nNR...,NS:ECLCTA
2023-04-05 12:58:53.344,2023-04-05 12:58:53.344000+00:00,Is SAP's Cloud Strategy Heading in the Right D...,urn:newsml:newswire.refinitiv.com:20230405:nNR...,NS:ZACKSC


In [16]:
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan

In [17]:
for idx, storyId in enumerate(df['storyId'].values):  #for each row in our df dataframe
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
        df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
        if sentA.sentiment.polarity >= 0.05: # attribute bucket to sentiment polartiy
            score = 'positive'
        elif  -.05 < sentA.sentiment.polarity < 0.05:
            score = 'neutral'
        else:
            score = 'negative'
        df['Score'].iloc[idx] = score #write score back to df
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score
2023-04-10 16:57:07.252,2023-04-10 16:57:07.252000+00:00,Fitch Assigns Qualtrics International Inc. Fir...,urn:newsml:newswire.refinitiv.com:20230410:nFI...,NS:FCH,0.078745,0.452093,positive
2023-04-07 02:55:46.751,2023-04-07 02:55:46.751000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA,0.000000,0.000000,neutral
2023-04-06 14:05:02.721,2023-04-06 14:05:02.721000+00:00,Augmentir Brings AI to SAP Plant Maintenance w...,urn:newsml:newswire.refinitiv.com:20230406:nPn...,NS:PRN,0.130408,0.405092,positive
2023-04-06 03:18:30.895,2023-04-06 03:18:30.895000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230406:nNR...,NS:ECLCTA,0.000000,0.000000,neutral
2023-04-05 12:58:53.344,2023-04-05 12:58:53.344000+00:00,Is SAP's Cloud Strategy Heading in the Right D...,urn:newsml:newswire.refinitiv.com:20230405:nNR...,NS:ZACKSC,0.125096,0.490952,positive


In [18]:
start = df['versionCreated'].min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
end = df['versionCreated'].max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
Minute = ek.get_timeseries(["SAP.N"], start_date=start, interval="minute")
Minute

SAP.N,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2023-03-13 13:31:00,114.49,114.31,114.31,114.49,30,8781
2023-03-13 13:32:00,114.49,114.23,114.47,114.39,19,2375
2023-03-13 13:33:00,114.39,114.24,114.39,114.24,10,1150
2023-03-13 13:34:00,114.44,114.29,114.37,114.44,15,1000
2023-03-13 13:35:00,114.34,114.19,114.34,114.24,14,1000
...,...,...,...,...,...,...
2023-04-11 20:00:00,126.54,126.45,126.48,126.53,51,3945
2023-04-11 20:01:00,126.54,126.54,126.54,126.54,2,24306
2023-04-11 20:11:00,126.54,126.54,126.54,126.54,1,<NA>


In [19]:
df['twoM'] = np.nan
df['fiveM'] = np.nan
df['tenM'] = np.nan
df['thirtyM'] = np.nan
df.head(2)

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-10 16:57:07.252,2023-04-10 16:57:07.252000+00:00,Fitch Assigns Qualtrics International Inc. Fir...,urn:newsml:newswire.refinitiv.com:20230410:nFI...,NS:FCH,0.078745,0.452093,positive,NaN,NaN,NaN,NaN
2023-04-07 02:55:46.751,2023-04-07 02:55:46.751000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN


In [20]:
df.tail()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-03-13 21:22:45.860,2023-03-13 21:22:45.860000+00:00,"SHAREHOLDER ALERT: Rigrodsky Law, P.A. Is Inve...",urn:newsml:newswire.refinitiv.com:20230313:nNF...,NS:NFC,0.013447,0.410795,neutral,NaN,NaN,NaN,NaN
2023-03-13 19:17:01.841,2023-03-13 19:17:01.841000+00:00,SHAREHOLDER ACTION ALERT: The Schall Law Firm ...,urn:newsml:newswire.refinitiv.com:20230313:nAC...,NS:ACSWIR,-0.142308,0.361538,negative,NaN,NaN,NaN,NaN
2023-03-13 18:55:27.000,2023-03-13 18:55:27+00:00,UPDATE 3-SAP-controlled Qualtrics accepts buyo...,urn:newsml:newswire.refinitiv.com:20230313:nL4...,NS:RTRS,0.112348,0.465745,positive,NaN,NaN,NaN,NaN
2023-03-13 11:31:46.739,2023-03-13 16:51:25.301000+00:00,Refinitiv Newscasts - U.S. Morning Call: Afte...,urn:newsml:newswire.refinitiv.com:20230313:nRT...,NS:RTRS,-0.103333,0.220000,negative,NaN,NaN,NaN,NaN
2023-03-13 14:51:44.579,2023-03-13 14:51:44.579000+00:00,SAP to Sell its Qualtrics Stake to Focus on Cl...,urn:newsml:newswire.refinitiv.com:20230313:nNR...,NS:ZACKSC,0.081134,0.505608,positive,NaN,NaN,NaN,NaN


In [21]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    sTime = df['versionCreated'][idx]
    sTime = sTime.replace(second=0,microsecond=0)
    try:
        t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
        df['twoM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=2))),3]/(t0)-1)*100)
        df['fiveM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=5))),3]/(t0)-1)*100)
        df['tenM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=10))),3]/(t0)-1)*100) 
        df['thirtyM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=30))),3]/(t0)-1)*100)
    except:
        pass
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-10 16:57:07.252,2023-04-10 16:57:07.252000+00:00,Fitch Assigns Qualtrics International Inc. Fir...,urn:newsml:newswire.refinitiv.com:20230410:nFI...,NS:FCH,0.078745,0.452093,positive,NaN,NaN,NaN,NaN
2023-04-07 02:55:46.751,2023-04-07 02:55:46.751000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230407:nNR...,NS:ECLCTA,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
2023-04-06 14:05:02.721,2023-04-06 14:05:02.721000+00:00,Augmentir Brings AI to SAP Plant Maintenance w...,urn:newsml:newswire.refinitiv.com:20230406:nPn...,NS:PRN,0.130408,0.405092,positive,NaN,NaN,NaN,NaN
2023-04-06 03:18:30.895,2023-04-06 03:18:30.895000+00:00,Maintenance For The Year 2023 Of The Software ...,urn:newsml:newswire.refinitiv.com:20230406:nNR...,NS:ECLCTA,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
2023-04-05 12:58:53.344,2023-04-05 12:58:53.344000+00:00,Is SAP's Cloud Strategy Heading in the Right D...,urn:newsml:newswire.refinitiv.com:20230405:nNR...,NS:ZACKSC,0.125096,0.490952,positive,NaN,NaN,NaN,NaN


In [23]:
grouped = df.groupby(['Score']).mean()
grouped

,Polarity,Subjectivity,twoM,fiveM,tenM,thirtyM
Score,,,,,,
negative,-0.094744,0.269551,NaN,NaN,NaN,NaN
neutral,0.013607,0.208390,NaN,NaN,NaN,NaN
positive,0.146846,0.399766,NaN,NaN,NaN,NaN
